In [ ]:
from heapq import heappop, heappush

# Directions and their movements
DIRECTIONS = ["up", "right", "down", "left"]
MOVES = {
    "up": (-1, 0),
    "down": (1, 0),
    "left": (0, -1),
    "right": (0, 1)
}

def find_positions(maze):
    """Locate the start (S) and end (E) positions in the maze."""
    start, end = None, None
    for x, row in enumerate(maze):
        for y, cell in enumerate(row):
            if cell == "S":
                start = (x, y)
            elif cell == "E":
                end = (x, y)
    return start, end

def reindeer_maze(file_path):
    """Find the lowest score to navigate the maze."""
    # Read the maze from the file
    with open(file_path, 'r') as file:
        maze = [line.strip() for line in file.readlines()]

    # Locate start and end points
    start, end = find_positions(maze)
    rows, cols = len(maze), len(maze[0])

    # Priority queue and visited set
    pq = []
    heappush(pq, (0, start[0], start[1], "right"))  # Start facing right
    visited = set()
    visited.add((start[0], start[1], "right"))

    while pq:
        score, x, y, direction = heappop(pq)

        # If we reach the end, return the score
        if (x, y) == end:
            return score

        # Explore moves
        # 1. Move forward
        dx, dy = MOVES[direction]
        nx, ny = x + dx, y + dy
        if 0 <= nx < rows and 0 <= ny < cols and maze[nx][ny] != "#":
            if (nx, ny, direction) not in visited:
                visited.add((nx, ny, direction))
                heappush(pq, (score + 1, nx, ny, direction))

        # 2. Rotate clockwise and counterclockwise
        for turn in [-1, 1]:  # -1 for counterclockwise, 1 for clockwise
            new_dir_idx = (DIRECTIONS.index(direction) + turn) % 4
            new_direction = DIRECTIONS[new_dir_idx]
            if (x, y, new_direction) not in visited:
                visited.add((x, y, new_direction))
                heappush(pq, (score + 1000, x, y, new_direction))

# Example Usage
file_path = "/content/drive/MyDrive/Personal Project/Advent of Code/2024/Day_16/input_16_12_2024.txt"  # Replace with the path to your maze file
print(reindeer_maze(file_path))


143572


In [ ]:
import heapq

def parse_maze(file_path):
    with open(file_path, 'r') as f:
        maze = [list(line.strip()) for line in f]

    start = end = None
    for y, row in enumerate(maze):
        for x, cell in enumerate(row):
            if cell == 'S':
                start = (x, y)
            elif cell == 'E':
                end = (x, y)

    return maze, start, end

def solve_maze(maze, start, end):
    directions = [(1, 0), (0, 1), (-1, 0), (0, -1)]  # East, South, West, North
    direction_cost = 1000

    def is_valid(x, y):
        return 0 <= x < len(maze[0]) and 0 <= y < len(maze) and maze[y][x] != '#'

    start_state = (*start, 0)  # (x, y, direction)
    pq = [(0, start_state)]  # (cost, (x, y, direction))
    visited = set()

    while pq:
        cost, (x, y, dir_idx) = heapq.heappop(pq)

        if (x, y, dir_idx) in visited:
            continue

        visited.add((x, y, dir_idx))

        # If we reach the end tile, return the cost
        if (x, y) == end:
            return cost

        # Try moving forward
        nx, ny = x + directions[dir_idx][0], y + directions[dir_idx][1]
        if is_valid(nx, ny):
            heapq.heappush(pq, (cost + 1, (nx, ny, dir_idx)))

        # Try rotating (clockwise and counterclockwise)
        for turn in [-1, 1]:
            new_dir_idx = (dir_idx + turn) % 4
            heapq.heappush(pq, (cost + direction_cost, (x, y, new_dir_idx)))

    return -1  # No solution found

# Input file path
file_path = "/content/drive/MyDrive/Personal Project/Advent of Code/2024/Day_16/input_16_12_2024.txt"  # Replace with the actual file path
maze, start, end = parse_maze(file_path)

# Solve the maze
minimum_score = solve_maze(maze, start, end)
print(f"The minimum score to solve the maze is: {minimum_score}")


The minimum score to solve the maze is: 143564


In [ ]:
import heapq

def parse_maze(file_path):
    with open(file_path, 'r') as f:
        maze = [list(line.strip()) for line in f]

    start = end = None
    for y, row in enumerate(maze):
        for x, cell in enumerate(row):
            if cell == 'S':
                start = (x, y)
            elif cell == 'E':
                end = (x, y)

    return maze, start, end

def solve_maze(maze, start, end):
    directions = [(1, 0), (0, 1), (-1, 0), (0, -1)]  # East, South, West, North
    direction_cost = 1000

    def is_valid(x, y):
        return 0 <= x < len(maze[0]) and 0 <= y < len(maze) and maze[y][x] != '#'

    start_state = (*start, 0, [])  # (x, y, direction, path)
    pq = [(0, start_state)]  # (cost, (x, y, direction, path))
    visited = {}
    best_paths = []
    min_cost = float('inf')

    while pq:
        cost, (x, y, dir_idx, path) = heapq.heappop(pq)

        if (x, y, dir_idx) in visited and visited[(x, y, dir_idx)] <= cost:
            continue

        visited[(x, y, dir_idx)] = cost

        # If we reach the end tile, check if it's a minimum-cost path
        if (x, y) == end:
            if cost < min_cost:
                min_cost = cost
                best_paths = [path + [(x, y)]]
            elif cost == min_cost:
                best_paths.append(path + [(x, y)])
            continue

        # Try moving forward
        nx, ny = x + directions[dir_idx][0], y + directions[dir_idx][1]
        if is_valid(nx, ny):
            heapq.heappush(pq, (cost + 1, (nx, ny, dir_idx, path + [(x, y)])))

        # Try rotating (clockwise and counterclockwise)
        for turn in [-1, 1]:
            new_dir_idx = (dir_idx + turn) % 4
            heapq.heappush(pq, (cost + direction_cost, (x, y, new_dir_idx, path)))

    # Collect all unique tiles from best paths
    optimal_tiles = set()
    for path in best_paths:
        optimal_tiles.update(path)

    return min_cost, optimal_tiles

def mark_optimal_tiles(maze, optimal_tiles):
    for y, row in enumerate(maze):
        for x, cell in enumerate(row):
            if (x, y) in optimal_tiles and cell not in ['S', 'E']:
                maze[y][x] = 'O'
    return maze

# Input file path
file_path = "/content/drive/MyDrive/Personal Project/Advent of Code/2024/Day_16/input_16_12_2024.txt"  # Replace with the actual file path
maze, start, end = parse_maze(file_path)

# Solve the maze
minimum_score, optimal_tiles = solve_maze(maze, start, end)

# Mark the optimal tiles in the maze
marked_maze = mark_optimal_tiles(maze, optimal_tiles)

# Output the results
print(f"The minimum score to solve the maze is: {minimum_score}")
print(f"Number of tiles part of the best paths: {len(optimal_tiles)}")
print("\n".join("".join(row) for row in marked_maze))


The minimum score to solve the maze is: 143564
Number of tiles part of the best paths: 565
#############################################################################################################################################
#...#.....#.......#...........#...#.............#.......#.........#.....................#.............#.........#.....#.......#.........#..E#
#.#.#.#.###.#.###.#####.#.#.#.#.#.#.###.#######.#######.#.#####.###.#.#######.###.#.#.#.#.#.###.#####.###.#####.#.###.#.###.#.#######.#.#.#O#
#.#...................#.....#...#.#.#.#.......#.........#.....#.....#.........#...#.#.#.#.#.........#.....#...#...#...#.#...#...#.....#...#O#
###.#.#.#.#####.#####.#.#.#######.#.#.#######.#######.#######.###########.#####.#.###.#.#.#########.#######.#.#.###.#####.#####.#.###.#####O#
#...#.#...#...#.#.#...#...#...#...#...#.....#.....#.#.#...#.............#...#...#.#...#...#.......#.....................#.#.....#...#OOOOO#O#
#.###.#####.#.#.#.#.###.#.###.#.#####.#.###.#####.#.#.#.#

In [10]:
import heapq
from collections import defaultdict

def parse_maze(file_path):
    with open(file_path, 'r') as f:
        maze = [list(line.strip()) for line in f]

    start = end = None
    for y, row in enumerate(maze):
        for x, cell in enumerate(row):
            if cell == 'S':
                start = (x, y)
            elif cell == 'E':
                end = (x, y)

    return maze, start, end

def solve_maze(maze, start, end):
    directions = [(1, 0), (0, 1), (-1, 0), (0, -1)]  # East, South, West, North
    direction_cost = 1000

    def is_valid(x, y):
        return 0 <= x < len(maze[0]) and 0 <= y < len(maze) and maze[y][x] != '#'

    pq = []  # Priority queue
    visited = defaultdict(lambda: float('inf'))
    best_paths = defaultdict(list)  # Track all paths to a node with minimum cost

    for dir_idx in range(4):  # Start from all initial directions
        heapq.heappush(pq, (0, start[0], start[1], dir_idx, [(start[0], start[1])]))

    min_cost = float('inf')

    while pq:
        cost, x, y, dir_idx, path = heapq.heappop(pq)

        if cost > visited[(x, y, dir_idx)]:
            continue
        visited[(x, y, dir_idx)] = cost

        # If we reach the end tile
        if (x, y) == end:
            if cost < min_cost:
                min_cost = cost
                best_paths[min_cost] = [path]
            elif cost == min_cost:
                best_paths[min_cost].append(path)
            continue

        # Try moving forward
        nx, ny = x + directions[dir_idx][0], y + directions[dir_idx][1]
        if is_valid(nx, ny):
            heapq.heappush(pq, (cost + 1, nx, ny, dir_idx, path + [(nx, ny)]))

        # Try rotating (clockwise and counterclockwise)
        for turn in [-1, 1]:
            new_dir_idx = (dir_idx + turn) % 4
            heapq.heappush(pq, (cost + direction_cost, x, y, new_dir_idx, path))

    # Collect all unique tiles and paths
    optimal_tiles = set()
    for path in best_paths[min_cost]:
        optimal_tiles.update(path)

    return min_cost, optimal_tiles, best_paths[min_cost]

def mark_paths(maze, paths):
    colors = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']  # For distinct colors

    for color_idx, path in enumerate(paths):
        color = colors[color_idx % len(colors)]
        for x, y in path:
            if maze[y][x] not in ['S', 'E']:
                maze[y][x] = color
    return maze

# Input file path
file_path = "/content/drive/MyDrive/Personal Project/Advent of Code/2024/Day_16/input_16_12_2024.txt"  # Replace with the actual file path
maze, start, end = parse_maze(file_path)

# Solve the maze
minimum_score, optimal_tiles, optimal_paths = solve_maze(maze, start, end)

# Mark the optimal paths in the maze
colored_maze = mark_paths(maze, optimal_paths)

# Output the results
print(f"The minimum score to solve the maze is: {minimum_score}")
print(f"Number of distinct tiles part of the best paths: {len(optimal_tiles)}")
print("\n".join("".join(row) for row in colored_maze))


The minimum score to solve the maze is: 142564
Number of distinct tiles part of the best paths: 593
#############################################################################################################################################
#...#.....#.......#...........#...#.............#.......#.........#.....................#.............#.........#.....#.......#.........#..E#
#.#.#.#.###.#.###.#####.#.#.#.#.#.#.###.#######.#######.#.#####.###.#.#######.###.#.#.#.#.#.###.#####.###.#####.#.###.#.###.#.#######.#.#.#7#
#.#...................#.....#...#.#.#.#.......#.........#.....#.....#.........#...#.#.#.#.#.........#.....#...#...#...#.#...#...#.....#...#7#
###.#.#.#.#####.#####.#.#.#######.#.#.#######.#######.#######.###########.#####.#.###.#.#.#########.#######.#.#.###.#####.#####.#.###.#####7#
#...#.#...#...#.#.#...#...#...#...#...#.....#.....#.#.#...#.............#...#...#.#...#...#.......#.....................#.#.....#...#77777#7#
#.###.#####.#.#.#.#.###.#.###.#.#####.#.###.####